## DIA 106: Crear un Dashboard con Gráficas en Tiempo Real usando Chart.js + Flask

Hoy vas a construir un dashboard visual en tu app Flask para mostrar datos históricos de predicción en tiempo real.
Utilizarás Chart.js, una librería de gráficos poderosa y fácil de integrar con Flask.

✅ ¿Qué aprenderás hoy?

📈 Mostrar gráficas de barras y líneas.

🔄 Actualizar los datos automáticamente desde Flask vía JSON.

🌍 Usar frontend + backend para monitoreo en tiempo real.

🧩 1. Instala Chart.js desde CDN
En tu archivo dashboard.html, coloca en el <head>:

html
Copiar
Editar
<script src="https://cdn.jsdelivr.net/npm/chart.js"></script>
🧩 2. Ruta Flask para el dashboard
python
Copiar
Editar
@app.route('/dashboard')
@login_required
def dashboard():
    return render_template("dashboard.html")
🧩 3. Ruta Flask para servir datos en JSON
python
Copiar
Editar
@app.route('/api/datos-dashboard')
@login_required
def datos_dashboard():
    registros = Registro.query.filter_by(user_id=current_user.id).order_by(Registro.fecha).all()

    labels = [r.fecha.strftime("%d-%m %H:%M") for r in registros]
    predicciones = [int(r.prediccion) for r in registros]

    return jsonify({
        "labels": labels,
        "datos": predicciones
    })
🧩 4. Template dashboard.html
html
Copiar
Editar
<!DOCTYPE html>
<html lang="es">
<head>
    <meta charset="UTF-8">
    <title>📊 Dashboard</title>
    <script src="https://cdn.jsdelivr.net/npm/chart.js"></script>
</head>
<body style="font-family:sans-serif; padding:20px;">
    <h2>📊 Historial de Predicciones</h2>
    <canvas id="grafico" width="800" height="400"></canvas>

    <script>
      const ctx = document.getElementById('grafico').getContext('2d');

      let chart = new Chart(ctx, {
        type: 'line',
        data: {
          labels: [],
          datasets: [{
            label: 'Predicciones',
            data: [],
            borderColor: 'blue',
            backgroundColor: 'lightblue',
            fill: false
          }]
        },
        options: {
          scales: {
            x: { title: { display: true, text: 'Fecha' } },
            y: { title: { display: true, text: 'Valor' }, beginAtZero: true }
          }
        }
      });

      async function actualizarDatos() {
        const res = await fetch("/api/datos-dashboard");
        const data = await res.json();

        chart.data.labels = data.labels;
        chart.data.datasets[0].data = data.datos;
        chart.update();
      }

      actualizarDatos();
      setInterval(actualizarDatos, 10000); // refresca cada 10 segundos
    </script>
</body>
</html>